In this notebook we use a demo transcript and store it into Pinecone and retrieve data according to a query.

Author: Sartaj, Madhuroopa

In [1]:
# # Have the `transcript.csv` file in the same directory as this notebook
# # Create a .env file with the following variables:
# OPENAI_API_KEY
# PINECONE_API_KEY
# PINECONE_ENVIRONMENT

In [4]:
pip install python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install openai


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/65.6 kB ? eta -:--:--
     ---------------------------------------- 65.6/65.6 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/58.3 kB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 58.3/58.3 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/224.7 kB ? eta -:--:--
   ------------------------------------- - 215.0/224.7 kB 12.8 MB/s eta 0:00:01
   ------------------------------------- - 215.0/224.7 kB 12.8 MB/s eta 0:00:01
   ---------------------------------------- 224.7/224.7 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/85.5 kB ? eta -:--:--
   -------------------------------------- - 81.9/85.5 kB ? eta -:--:--
   -------------------------------------- - 81.9/85.5 kB ? eta -:--:--
   ------------

In [12]:
pip install langchain  


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/798.0 kB ? eta -:--:--
   ----------- ---------------------------- 225.3/798.0 kB 6.9 MB/s eta 0:00:01
   ------------------------ -------------- 491.5/798.0 kB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 798.0/798.0 kB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/364.8 kB ? eta -:--:--
   --------------------------------------- 364.8/364.8 kB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------------------------------------- - 1.5/1.5 MB 47.2 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 32.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/216.5 kB ? eta -:--:--
   --------------------------------------- 216.5/216.5 kB 12.9 MB/s eta 0:00:00
   -----------------

In [20]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 217.9 kB/s eta 0:00:01
     -------------------------------------  41.0/42.0 kB 279.3 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 202.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/786.4 kB ? eta -:--:--
   --- ------------------------------------ 61.4/786.4 kB 1.7 MB/s eta 0:00:01
   ---------------- ----------------------- 317.4/786.4 kB 3.9 MB/s eta 0:00:01
   ---------------------------------------  778.2/786.4 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 786.4/786.4 kB 6.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/269.5 kB ? eta -:--:--
   -

In [11]:
pip install pinecone-client

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/179.4 kB ? eta -:--:--
   ------------- -------------------------- 61.4/179.4 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 179.4/179.4 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/300.4 kB ? eta -:--:--
   ---------------------------------------- 300.4/300.4 kB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/62.5 kB ? eta -:--:--
   ---------------------------------------- 62.5/62.5 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import dotenv
dotenv.load_dotenv()

True

Read Data and Pre-process

In [2]:


transcript = pd.read_csv('skiing.csv')
transcript.head()

,start_time,speaker_label,text
0,0.0000,Speaker 1,"Hello everyone, thanks for joining. Let's disc..."
1,0.0025,Speaker 2,I'm excited! Where should we go skiing?
2,0.0050,Speaker 1,I was thinking of heading to the mountains. Th...
3,0.0075,Speaker 3,That sounds great. What time should we start o...
4,0.0100,Speaker 1,"I suggest we leave early in the morning, aroun..."


In [10]:
transcript.dropna(inplace=True)
transcript.drop(['Unnamed: 0', 'end_time' ], axis=1, inplace=True)
transcript.head()

KeyError: "['Unnamed: 0', 'end_time'] not found in axis"

In [3]:
import warnings
warnings.filterwarnings("ignore")
to_remove=[]
for i, record in transcript.iterrows():
    if i < len(transcript)-1:
        if transcript.speaker_label[i]== transcript.speaker_label[i+1]:
            transcript['text'][i]=transcript['text'][i]+" "+transcript['text'][i+1]
            to_remove.append(i+1)

transcript = transcript.drop(to_remove).reset_index(drop=True)

In [4]:
transcript.head()

,start_time,speaker_label,text
0,0.0000,Speaker 1,"Hello everyone, thanks for joining. Let's disc..."
1,0.0025,Speaker 2,I'm excited! Where should we go skiing?
2,0.0050,Speaker 1,I was thinking of heading to the mountains. Th...
3,0.0075,Speaker 3,That sounds great. What time should we start o...
4,0.0100,Speaker 1,"I suggest we leave early in the morning, aroun..."


Creating Embeddings

In [5]:
import os
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

In [6]:
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

C:\Users\madhu\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [7]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed.embed_documents(texts)
len(res), len(res[0]) # (x,1536) 1536 is the embedding size

(2, 1536)

In [8]:
def create_embedding(text):
    return embed.embed_documents([text])[0]

# get first row of transcript
test_embedding_function = create_embedding(transcript.iloc[0]['text'])
len(test_embedding_function) # 1 X 1536


1536

Pinecone

In [9]:
import pinecone
import time

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY') or 'PINECONE_API_KEY'
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'
index_name = 'langchain-retrieval-transcript'
namespace = 'meeting_topic'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT)

In [10]:
print(pinecone.list_indexes())
for index in pinecone.list_indexes():
    print(pinecone.delete_index(index))

['langchain-retrieval-transcript']
None


In [11]:
# create index
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        metric='euclidean',
        dimension=len(res[0])  # model_name = 'text-embedding-ada-002'; 1536 dim of text-embedding-ada-002
    )   
    
while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)

In [12]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [25]:


# index.delete(delete_all=True, namespace=namespace)

Inserting data into Pinecone

In [13]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 10
texts = []
metadatas = []
meeting_id = 1

for i, record in tqdm(transcript.iterrows()):
    # first get metadata fields for this record
    metadata = {
        'speaker': record['speaker_label'],
        'start_time': round(record['start_time'], 4), # limit to 4 decimal places 
        'meeting_id': meeting_id,
        'text': record['text'], # Storing the text in the metadata for now, later we'd need to decode it from vectors
    }

    record_texts = record['text']

    texts.append(record_texts)
    metadatas.append(metadata)

    # print(texts)
    # print(metadatas)
    
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas), namespace=namespace)
        texts = []
        metadatas = []
        meeting_id += 1

# add any remaining texts
if len(texts) > 0:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))
    
time.sleep(5)    

0it [00:00, ?it/s]

17it [00:00, 23.10it/s]


In [16]:
index.describe_index_stats()   

{'dimension': 1536,
 'index_fullness': 0.00017,
 'namespaces': {'': {'vector_count': 7}, 'meeting_topic': {'vector_count': 10}},
 'total_vector_count': 17}

Querying Pinecone DB

In [102]:
# Need to look up about using LangChain for retrieval

In [17]:
import json
query = "What was the skiing plan?"
downstr_response = index.query(
    vector=embed.embed_documents([query])[0],
    namespace=namespace,
    top_k=30,
    include_metadata=True,
)

print(downstr_response)


{'matches': [{'id': '6f08626f-bbdc-42b3-bd7a-af7450eaf6bf',
              'metadata': {'meeting_id': 1.0,
                           'speaker': 'Speaker 1',
                           'start_time': 0.0,
                           'text': "Hello everyone, thanks for joining. Let's "
                                   'discuss our skiing plans for this '
                                   'weekend.'},
              'score': 0.305710793,
              'values': []},
             {'id': '5d6890eb-e544-4ebf-b151-d321051c669e',
              'metadata': {'meeting_id': 1.0,
                           'speaker': 'Speaker 2',
                           'start_time': 0.0025,
                           'text': "I'm excited! Where should we go skiing?"},
              'score': 0.31223309,
              'values': []},
             {'id': '430e338a-4420-4a2a-b422-80fbc139af09',
              'metadata': {'meeting_id': 1.0,
                           'speaker': 'Speaker 2',
                          

In [28]:
# Extract relevant information from the metadata
results = []
for hit in downstr_response["matches"]:
    metadata = hit["metadata"]
    start_time = metadata["start_time"]
    speaker_label = metadata["speaker"]
    text = metadata["text"]

# Get the text spoken +30 and -30 seconds before and after the identified timestamp
    start_time_up = max(0, start_time - 0.03)
    start_time_down = start_time + 0.03

# Query the Pinecone database to get the relevant text within the specified time window
    relevant_texts_response = index.query(
        vector = embed.embed_documents([metadata["text"]])[0],
        filter={"start_time": {"$gte": start_time_up, "$lte": start_time_down}},
        top_k=2,  
        namespace=namespace,
        include_metadata=True
    )

    relevant_texts = [hit["metadata"]["text"] for hit in relevant_texts_response["matches"]]

# Add the relevant information to the results
    result = {
    "start_time_up": start_time_up,
    "start_time": start_time,
    "start_time_down": start_time_down,
    "speaker_label": speaker_label,
    "text": text,
    "relevant_texts": relevant_texts,
    }
    results.append(result)

print(results)

[{'start_time_up': 0, 'start_time': 0.0, 'start_time_down': 0.03, 'speaker_label': 'Speaker 1', 'text': "Hello everyone, thanks for joining. Let's discuss our skiing plans for this weekend.", 'relevant_texts': ["Hello everyone, thanks for joining. Let's discuss our skiing plans for this weekend.", "I'm excited! Where should we go skiing?"]}, {'start_time_up': 0, 'start_time': 0.0025, 'start_time_down': 0.0325, 'speaker_label': 'Speaker 2', 'text': "I'm excited! Where should we go skiing?", 'relevant_texts': ["I'm excited! Where should we go skiing?", "Perfect! I'll make sure to pack all the necessary gear for skiing."]}, {'start_time_up': 0, 'start_time': 0.0125, 'start_time_down': 0.042499999999999996, 'speaker_label': 'Speaker 2', 'text': "Perfect! I'll make sure to pack all the necessary gear for skiing.", 'relevant_texts': ["Perfect! I'll make sure to pack all the necessary gear for skiing.", "I'm excited! Where should we go skiing?"]}, {'start_time_up': 0, 'start_time': 0.005, 'st

In [29]:
import json
formatted_results = json.dumps(results, indent=4)

print(formatted_results)

[
    {
        "start_time_up": 0,
        "start_time": 0.0,
        "start_time_down": 0.03,
        "speaker_label": "Speaker 1",
        "text": "Hello everyone, thanks for joining. Let's discuss our skiing plans for this weekend.",
        "relevant_texts": [
            "Hello everyone, thanks for joining. Let's discuss our skiing plans for this weekend.",
            "I'm excited! Where should we go skiing?"
        ]
    },
    {
        "start_time_up": 0,
        "start_time": 0.0025,
        "start_time_down": 0.0325,
        "speaker_label": "Speaker 2",
        "text": "I'm excited! Where should we go skiing?",
        "relevant_texts": [
            "I'm excited! Where should we go skiing?",
            "Perfect! I'll make sure to pack all the necessary gear for skiing."
        ]
    },
    {
        "start_time_up": 0,
        "start_time": 0.0125,
        "start_time_down": 0.042499999999999996,
        "speaker_label": "Speaker 2",
        "text": "Perfect! I'll make